In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from childes_mi.utils.paths import DATA_DIR, FIGURE_DIR
from childes_mi.utils.general import flatten,save_fig

In [3]:
from childes_mi.information_theory import model_fitting as mf

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
MI_DF = pd.read_pickle(DATA_DIR/'mi/zebra_fish_1000.pickle')

In [6]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances
0,"[11.122280319221097, 10.9123149216584, 10.8559...","[0.005183830214899432, 0.005129859794221152, 0...","[10.810847766607214, 10.810677673361408, 10.81...","[0.005150801831295093, 0.0051564790406694195, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [7]:
MI_DF = MI_DF.assign(**{i:np.nan for i in ['exp_results', 'pow_results', 'concat_results',
     'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
     'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak']})
MI_DF['curvature'] = MI_DF['curvature'].astype(object)

In [8]:
n = 100 # max distance for computation
for idx, row in tqdm(MI_DF.iterrows(), total=len(MI_DF)):
    # get signal
    sig = np.array(row.MI-row.shuff_MI)
    distances = row.distances
    sig = sig
    
    # fit models
    results_power, results_exp, results_pow_exp, best_fit_model = mf.fit_models(distances, sig)
    
    # get fit results
    R2_exp, R2_concat, R2_power, AICc_exp, \
        AICc_pow, AICc_concat = mf.fit_results(sig, distances, 
                                              results_exp, results_power,
                                              results_pow_exp)
    
    
    
    
    # get model y
    distances_mod = np.logspace(0,np.log10(n), base=10, num=1000)
    if best_fit_model == 'pow_exp':
        y_model = mf.get_y(mf.pow_exp_decay, results_pow_exp, distances_mod)
    elif best_fit_model == 'exp':
        y_model = mf.get_y(mf.exp_decay, results_exp, distances_mod)
    elif best_fit_model == 'pow':
        y_model = mf.get_y(mf.powerlaw_decay, results_power, distances_mod)
    
    # get curvature of model_y
    curvature_model = mf.curvature(np.log(y_model))
    
    # if the best fit model is pow_exp, then grab the min peak
    if best_fit_model == 'pow_exp':
        # get peaks of curvature
        peaks = np.where((
            (curvature_model[:-1] < curvature_model[1:])[1:] & (curvature_model[1:] < curvature_model[:-1])[:-1]
        ))
        min_peak = peaks[0][0]
    else:
        min_peak = np.nan

    # get save model fit results to MI_DF
    MI_DF.loc[idx, np.array(['exp_results', 'pow_results', 'concat_results',
                         'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
                         'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak'])] = [
        results_exp, results_power, results_pow_exp,
        R2_exp, R2_concat, R2_power, AICc_exp,
        AICc_concat, AICc_pow, best_fit_model,
        curvature_model, min_peak
    ]


In [9]:
MI_DF

,MI,MI_var,shuff_MI,shuff_MI_var,distances,exp_results,pow_results,concat_results,R2_exp,R2_concat,R2_power,AICc_exp,AICc_concat,AICc_power,bestfitmodel,curvature,min_peak
0,"[11.122280319221097, 10.9123149216584, 10.8559...","[0.005183830214899432, 0.005129859794221152, 0...","[10.810847766607214, 10.810677673361408, 10.81...","[0.005150801831295093, 0.0051564790406694195, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",<lmfit.minimizer.MinimizerResult object at 0x7...,<lmfit.minimizer.MinimizerResult object at 0x7...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.897209,0.979373,0.968537,-4914.072776,-6516.146249,-6097.964376,pow_exp,"[-1.0434379676410966e-05, -1.5666134278998892e...",67.0


In [10]:
labels = ['Zebrafish']

### Calculate R2

In [11]:
R2 = MI_DF[['R2_exp', 'R2_concat', 'R2_power']]
R2.columns = ['exp', 'combined', 'power-law']
R2.index = labels
R2 = R2.T
R2

,Zebrafish
exp,0.897209
combined,0.979373
power-law,0.968537


### AICc

In [12]:
AICcs = MI_DF[['AICc_exp', 'AICc_concat', 'AICc_power']]
AICcs.columns = ['exp', 'combined.', 'power-law']
AICcs.index = labels
AICcs = AICcs.T
AICcs

,Zebrafish
exp,-4914.072776
combined.,-6516.146249
power-law,-6097.964376


#### $\Delta$AIC

In [13]:
delta_AICcs = AICcs.T - np.repeat(np.min(AICcs.T.values, axis=1),3).reshape(len(labels),3)
delta_AICcs = delta_AICcs.T
delta_AICcs

,Zebrafish
exp,1602.073473
combined.,0.000000
power-law,418.181873


### relative likelihood

In [14]:
relative_likelihoods = mf.relative_likelihood(delta_AICcs)
relative_likelihoods

,Zebrafish
exp,0.000000e+00
combined.,1.000000e+00
power-law,1.559409e-91


### relative probability of models

In [15]:
prob_models = mf.Prob_model_Given_data_and_models(relative_likelihoods)
prob_models

,Zebrafish
exp,0.000000e+00
combined.,1.000000e+00
power-law,1.559409e-91


#### Join tables

In [16]:
AICcs['superlabel'] = 'AICc'
relative_likelihoods['superlabel'] = 'Relative likelihood'
R2['superlabel'] = '$r^2$'
prob_models['superlabel'] = 'Relative probability'

In [17]:
results_table = pd.concat([
    AICcs,
    R2,
    relative_likelihoods,
    prob_models
]).round(3).replace(0, '<0.001').replace(1, '>0.999')
results_table[''] = results_table.index
results_table.set_index(['superlabel', ''], inplace=True)
results_table

Zebrafish
superlabel                              
AICc                 exp        -4914.07
                     combined.  -6516.15
                     power-law  -6097.96
$r^2$                exp           0.897
                     combined      0.979
                     power-law     0.969
Relative likelihood  exp          <0.001
                     combined.    >0.999
                     power-law    <0.001
Relative probability exp          <0.001
                     combined.    >0.999
                     power-law    <0.001

In [18]:
results_string = results_table.to_latex(bold_rows=True, escape=False)\
      .replace('>', '$>$')\
      .replace('<', '$<$')\
      .replace('superlabel', '')\
     .replace('\n\\textbf', '\n\midrule\n\\textbf')
print(results_string)

\begin{tabular}{lll}
\toprule
                     &           & Zebrafish \\
\midrule
\textbf{} & {} &           \\
\midrule
\midrule
\textbf{AICc} & \textbf{exp} &  -4914.07 \\
                     & \textbf{combined.} &  -6516.15 \\
                     & \textbf{power-law} &  -6097.96 \\
\midrule
\textbf{$r^2$} & \textbf{exp} &     0.897 \\
                     & \textbf{combined} &     0.979 \\
                     & \textbf{power-law} &     0.969 \\
\midrule
\textbf{Relative likelihood} & \textbf{exp} &    $<$0.001 \\
                     & \textbf{combined.} &    $>$0.999 \\
                     & \textbf{power-law} &    $<$0.001 \\
\midrule
\textbf{Relative probability} & \textbf{exp} &    $<$0.001 \\
                     & \textbf{combined.} &    $>$0.999 \\
                     & \textbf{power-law} &    $<$0.001 \\
\bottomrule
\end{tabular}

